In [ ]:
import pandas as pd
import requests

In [50]:
symbol = "EMAAR"
exchange = "DFM" # ADSM or DFM, ADSM = ADX
session_id = "3cc6f101-6064-49a4-8962-b7126d7f63d6"
start = "1900-01-01T00:00:00.000Z"
end = "2100-01-31T20:00:00.000Z"
interval = "1" # 1 days
additional_slug = "1753943650537"

url = f"https://mobile.intlsecurities.ae/FITDataFeedServiceGateway/DataFeedService.asmx/datafeedDFN?session={session_id}&symbol={symbol},{exchange}&period=day&from={start}&to={end}&interval={1}${additional_slug}"

In [49]:
def fetch_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status() 

        data = response.json()
        df = pd.DataFrame(data)

        if 'DT' in df.columns:
            df.rename(columns={'DT': 'Date'}, inplace=True)
            df['Date'] = pd.to_datetime(df['Date']).dt.date
            df = df.set_index('Date')

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
    except ValueError:
        print("Error: Could not decode JSON from the response.")
    
    return df, data

df, data = fetch_data(url)
print(data[0:5])
print(df.head())

[{'DT': '2025-07-31T00:00:00.000Z', 'Open': 15.45, 'High': 15.5, 'Low': 15.4, 'Close': 15.5, 'Volume': 956461}, {'DT': '2025-07-30T00:00:00.000Z', 'Open': 15.45, 'High': 15.5, 'Low': 15.25, 'Close': 15.45, 'Volume': 10185647}, {'DT': '2025-07-29T00:00:00.000Z', 'Open': 15.5, 'High': 15.6, 'Low': 15.15, 'Close': 15.45, 'Volume': 14328082}, {'DT': '2025-07-25T00:00:00.000Z', 'Open': 15.3, 'High': 15.9, 'Low': 15.3, 'Close': 15.7, 'Volume': 16099431}, {'DT': '2025-07-23T00:00:00.000Z', 'Open': 15.0, 'High': 15.1, 'Low': 14.9, 'Close': 15.1, 'Volume': 10311380}]
             Open  High    Low  Close    Volume
Date                                           
2025-07-31  15.45  15.5  15.40  15.50    956461
2025-07-30  15.45  15.5  15.25  15.45  10185647
2025-07-29  15.50  15.6  15.15  15.45  14328082
2025-07-25  15.30  15.9  15.30  15.70  16099431
2025-07-23  15.00  15.1  14.90  15.10  10311380


In [ ]:
trans_log = pd.read_csv('raw-data/IntlSecurities_July2025.csv')
symbol_market_pair = trans_log[['Symbol', 'Market']].dropna().drop_duplicates()

for _, row in symbol_market_pair.iterrows():
    symbol = row['Symbol']
    exchange = row['Market']
    url = f"https://mobile.intlsecurities.ae/FITDataFeedServiceGateway/DataFeedService.asmx/datafeedDFN?session={session_id}&symbol={symbol},{exchange}&period=day&from={start}&to={end}&interval={1}${additional_slug}"
    df, _ = fetch_data(url)
    df.to_csv(f"../data/cache/{symbol}-{exchange}.csv", index=False)

![test.png](/Users/rehabnaeem/Developer/investment-portfolio-test/tools/IntlSecurities-data-scrapping.png)